<a href="https://colab.research.google.com/github/Dinhthixuanbinh/Htech_note/blob/clean_data_branch/Detectron2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!python -m pip install pyyaml==5.1
import sys, os, distutils.core
# Note: This is a faster way to install detectron2 in Colab, but it does not include all functionalities (e.g. compiled operators).
# See https://detectron2.readthedocs.io/tutorials/install.html for full installation instructions
!git clone 'https://github.com/facebookresearch/detectron2'
dist = distutils.core.run_setup("./detectron2/setup.py")
!python -m pip install {' '.join([f"'{x}'" for x in dist.install_requires])}
sys.path.insert(0, os.path.abspath('./detectron2'))

# Properly install detectron2. (Please do not install twice in both ways)
# !python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.2/274.2 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyyaml: filename=PyYAML-5.1-cp310-cp310-linux_x86_64.whl size=44091 sha256=c33a1d1befb4b4a940984d59ae9bf83a43ffb4f9669967417d9f97363fb9a42f
  Stored in directory: /root/.cache/pip/wheels/70/83/31/975b737609aba39a4099d471d5684141c1fdc3404f97e7f68a
Successfully built pyyaml
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 6.0.1
    Uninstalling PyYAML-6.0.1:
      Successfully uninstalled PyYAML-6.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dask 2022.12.1 requires pyyaml>=5.3.1, but you have pyyaml 5.1 which is incompatible.
flax 0.7.0 requires PyYAML>=5.4.1, but you have pyyaml 5.1 which is incompatible.
Cloning into 'detectron2'...
remote: Enumerating objects: 15180, done.
remote: Counting ob

In [ ]:
import torch, detectron2
from detectron2 import model_zoo
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0
torch:  2.0 ; cuda:  cu118
detectron2: 0.6


In [ ]:
!git clone https://github.com/CaseDrive/publaynet-models.git

Cloning into 'publaynet-models'...
remote: Enumerating objects: 339, done.
remote: Counting objects: 100% (339/339), done.
remote: Compressing objects: 100% (247/247), done.
remote: Total 339 (delta 85), reused 339 (delta 85), pack-reused 0
Receiving objects: 100% (339/339), 7.93 MiB | 17.45 MiB/s, done.
Resolving deltas: 100% (85/85), done.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import cv2
import numpy
from PIL import Image as image_main
from detectron2 import model_zoo
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor
import os, shutil


# Model, threshold score, class labels, and example image - be sure to replace with your own
# image_path = '/content/publaynet-models/usage_example/PMC1500815_00002.jpg'
image_path = '/content/test.jpg'
model_path = '/content/drive/MyDrive/Detectron2/faster_rcnn_R_101_FPN_3x/model_final.pth'
model_zoo_config_name = 'COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml'
prediction_score_threshold = 0.7
class_labels = ['text', 'title', 'list', 'table', 'figure']

# Detectron config
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file(model_zoo_config_name))
cfg.MODEL.WEIGHTS = model_path
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = prediction_score_threshold
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 5

# Detectron predictor
predictor = DefaultPredictor(cfg)

# Open image, and run predictor
image_pil = image_main.open(image_path)
image_cv = cv2.cvtColor(numpy.array(image_pil), cv2.COLOR_RGB2BGR)
outputs = predictor(image_cv)

# Debug outputs
instances = outputs["instances"].to("cpu")
pred_boxes = instances.pred_boxes
scores = instances.scores
pred_classes = instances.pred_classes

image = cv2.imread(image_path)
os.makedirs('result', exist_ok=True)

folder = '/content/result'
for filename in os.listdir(folder):
    file_path = os.path.join(folder, filename)
    try:
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path)
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)
    except Exception as e:
        print('Failed to delete %s. Reason: %s' % (file_path, e))

for i in range(0, len(pred_boxes)):
    box = pred_boxes[i].tensor.numpy()[0]
    score = round(float(scores[i].numpy()), 4)
    label_key = int(pred_classes[i].numpy())
    label = class_labels[label_key]
    x = int(box[0])
    y = int(box[1])
    w = int(box[2] - box[0])
    h = int(box[3] - box[1])

    x_1 = int(box[0])
    y_1 = int(box[1])
    x_2 = int(box[2])
    y_2 = int(box[3])
    extracted_region = image[y_1:y_2, x_1:x_2]
    cv2.imwrite(f'result/{label}_{i}.jpg', extracted_region)

    print('Detected object of label=' + str(label) + ' with score=' + str(score) + ' and in box={x=' + str(x) + ', y=' + str(y) + ', w=' + str(w) + ', h=' + str(h) + '}')


Detected object of label=table with score=0.9982 and in box={x=14, y=404, w=697, h=189}
Detected object of label=table with score=0.9973 and in box={x=18, y=186, w=687, h=182}
Detected object of label=title with score=0.9588 and in box={x=18, y=657, w=383, h=17}
Detected object of label=table with score=0.9323 and in box={x=26, y=143, w=641, h=495}
Detected object of label=title with score=0.8851 and in box={x=14, y=625, w=48, h=15}
Detected object of label=text with score=0.8021 and in box={x=16, y=61, w=663, h=36}
Detected object of label=title with score=0.7325 and in box={x=17, y=150, w=146, h=15}
